# Part One

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
#Libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from matplotlib.pyplot import figure
import pyodbc 
import glob
import copy 

# Comparison Functions

## MGRA Level Data

In [24]:
first_ID_processed, first_ID_unprocessed, first_ID_features = download_DS_data(first_ID, jur_level)

### Downloading  the second_ID Data

In [ ]:
second_ID_processed, second_ID_unprocessed, second_ID_features = download_DS_data(second_ID, jur_level)

### Concatenate both DS dataframes

In [156]:
def concat_dfs(comparison_first_ID_processed_data, comparison_second_ID_processed_data):
    # Added geozone to merge keys to account for mgra's in multiple jurisdictions (or other geographical levels)
    first_second_ID_comparison = comparison_first_ID_processed_data.merge(
        comparison_second_ID_processed_data,
        how='left',
        left_on=[f'mgra_{first_ID}',
                 f'year_{first_ID}',
                 f'geozone_{first_ID}'],
        right_on=[f'mgra_{second_ID}',
                 f'year_{second_ID}',
                 f'geozone_{second_ID}'])
    
    # Clean green combined
    first_second_ID_comparison = first_second_ID_comparison.drop([f'mgra_{second_ID}', f'year_{second_ID}', f'geozone_{second_ID}'], axis=1)
    first_second_ID_comparison = first_second_ID_comparison.rename(columns={f'mgra_{first_ID}': 'mgra', f'year_{first_ID}': 'year', f'geozone_{first_ID}': 'geozone'})
    
    # Because we're summing, if using series 14 data, mgra's in multiple jurisdictions will be counted multiple times
    first_second_ID_comparison = first_second_ID_comparison.groupby(['mgra', 'year']).sum()
    
    #first_second_ID_comparison.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison")
    
    return first_second_ID_comparison

In [157]:
mgra_both = concat_dfs(first_ID_processed, second_ID_processed)
mgra_both.head()

taz_DS35  hs_DS35  hs_sf_DS35  hs_mf_DS35  hs_mh_DS35  hh_DS35  \
mgra year                                                                   
1    2016      3331       19          19           0           0       18   
     2018      3331       19          19           0           0       18   
     2020      3331       19          19           0           0       18   
     2023      3331       20          20           0           0       18   
     2025      3331       20          20           0           0       18   

           hh_sf_DS35  hh_mf_DS35  hh_mh_DS35  gq_civ_DS35  ...  \
mgra year                                                   ...   
1    2016          18           0           0            0  ...   
     2018          18           0           0            0  ...   
     2020          18           0           0            0  ...   
     2023          18           0           0            0  ...   
     2025          18           0           0            0  ...   

           midpriceroom_DS41  upscaleroom_DS41  hotelroomtotal_DS41  \
mgra year                                                             
1    2016                  0                 0                    0   
     2018                  0                 0                    0   
     2020                  0                 0                    0   
     2023                  0                 0                    0   
     2025                  0                 0                    0   

           luz_id_DS41  truckregiontype_DS41  district27_DS41  \
mgra year                                                       
1    2016           95                     1               27   
     2018           95                     1               27   
     2020           95                     1               27   
     2023           95                     1               27   
     2025           95                     1               27   

           milestocoast_DS41  acres_DS41  effective_acres_DS41  \
mgra year                                                        
1    2016             3.7997   16.615444             12.961482   
     2018             3.7997   16.615444             12.961482   
     2020             3.7997   16.615444             12.961482   
     2023             3.7997   16.615444             12.961482   
     2025             3.7997   16.615444             12.961482   

           land_acres_DS41  
mgra year                   
1    2016        16.615444  
     2018        16.615444  
     2020        16.615444  
     2023        16.615444  
     2025        16.615444  

[5 rows x 206 columns]

## CPA level Data

In [158]:
def cpa_aggregation(first_ID_df, second_ID_df, cpa_level):
    
    # Adding SQl Data (CPA) to first_id_df
    comparison_first_ID_processed_data_cpa = first_ID_df.merge(cpa_level, how='left', on='mgra')
    comparison_first_ID_processed_data_cpa = comparison_first_ID_processed_data_cpa[comparison_first_ID_processed_data_cpa['geozone'] != '*Not in a CPA*']

    # Adding SQl Data (CPA) to second_id_df
    comparison_second_ID_processed_data_cpa = second_ID_df.merge(cpa_level, how='left', on='mgra')
    comparison_second_ID_processed_data_cpa = comparison_second_ID_processed_data_cpa[comparison_second_ID_processed_data_cpa['geozone'] != '*Not in a CPA*']

    # Merge first_id_df and second_id_df together on mgra, year, and geozone
    comparison_processed_data_cpa = comparison_first_ID_processed_data_cpa.merge(comparison_second_ID_processed_data_cpa, how='inner', on=['mgra', 'year', 'geozone'], suffixes=[f'_{first_ID}', f'_{second_ID}'])

    # Drop the MGRA column because it isn't really a quantitative value
    comparison_processed_data_cpa = comparison_processed_data_cpa.drop('mgra', axis=1)

    # Aggregate the sum of features by geozone and year
    comparison_processed_data_cpa = comparison_processed_data_cpa.groupby(['geozone', 'year']).sum()

    # Rename index (geozone -> cpa)
    comparison_processed_data_cpa.index.names = ['cpa', 'year']
    
    #comparison_processed_data_cpa.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison_cpa")

    return comparison_processed_data_cpa

In [159]:
cpa_both = cpa_aggregation(first_ID_df, second_ID_df, cpa_level)
cpa_both.head()

NameError: name 'first_ID_df' is not defined

## Jurisdiction level Data

In [162]:
def jur_aggregation(first_ID_df, second_ID_df, jur_level):
    # Adding SQl Data (Jurisdiction) to first_id_df
    comparison_first_ID_processed_data_jur = first_ID_df.merge(jur_level, how='left', on='mgra')
    
    # Adding SQl Data (Jurisdiction) to second_id_df
    comparison_second_ID_processed_data_jur = second_ID_df.merge(jur_level, how='left', on='mgra')
    
    # Merge first_id_df and second_id_df together on mgra, year, and geozone
    comparison_processed_data_jur = comparison_first_ID_processed_data_jur.merge(comparison_second_ID_processed_data_jur, how='inner', on=['mgra', 'year', 'geozone'], suffixes=[f'_{first_ID}', f'_{second_ID}'])
    
    # Drop the MGRA column because it isn't really a quantitative value
    comparison_processed_data_jur = comparison_processed_data_jur.drop('mgra', axis=1)
    
    # Aggregate the sum of features by geozone and year
    comparison_processed_data_jur = comparison_processed_data_jur.groupby(['geozone', 'year']).sum()
    
    # Rename index (geozone -> jurisdiction)
    comparison_processed_data_jur.index.names = ['jurisdiction', 'year']
    
    #comparison_processed_data_jur.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison_jurisdiction")
    
    return comparison_processed_data_jur

In [ ]:
jur_both = jur_aggregation(first_ID_df, second_ID_df, jur_level)
jur_both.head()

## Creating Diff File for all Geo Levels

In [164]:
def non_shared_features(features_first_ID, features_second_ID):
    """
    (Comparison only)
    Identifies non-shared features between two different DS_ID's.
    """
    # In case we want to display non-shared features
    return list(first_ID_unique ^ set(features_second_ID))

In [163]:
def create_diff(features_first_ID, features_second_ID, first_second_ID_comparison):
    """
    (Comparison only)
    """
    # Finding features common to both DSID data frames
    first_ID_unique = set(features_first_ID)
    intersection = first_ID_unique.intersection(features_second_ID)
    shared_features = list(intersection)
    
    # Calculate diff values between the two DS_ID's
    diff_df = pd.DataFrame()

    # NOTE: Subtracts second DS ID from first DS ID. If negative, then second DS ID was greater than first DS ID.
    for column in shared_features:
        diff_df[f'{column}_diff'] = first_second_ID_comparison[f'{column}_{first_ID}'] - first_second_ID_comparison[f'{column}_{second_ID}']
    
    #diff_df.to_csv(f"{output_path}/{first_ID}_{second_ID}_complete_diff")
    
    return diff_df

In [ ]:
# Create diff at mgra level
mgra_diff = create_diff(first_ID_features, second_ID_features, mgra_both)
mgra_diff.head()

In [ ]:
# Create diff at cpa level
cpa_diff = create_diff(first_ID_features, second_ID_features, cpa_both)
cpa_diff.head()

In [ ]:
# Create diff at jurisdiction level
jur_diff = create_diff(first_ID_features, second_ID_features, jur_both)
jur_diff.head()

## Region level Data

In [165]:
def region_aggregation(first_ID_df, second_ID_df):
    # Merge first_id_df and second_id_df together on mgra and year
    comparison_processed_data_reg = first_ID_df.merge(second_ID_df, how='inner', on=['mgra', 'year'], suffixes=[f'_{first_ID}', f'_{second_ID}'])
    
    # Aggregate the sum of features by year
    comparison_processed_data_reg = comparison_processed_data_reg.groupby('year').sum()
    
    # Drop the MGRA column because it isn't really a quantitative value
    comparison_processed_data_reg = comparison_processed_data_reg.drop('mgra', axis=1)
    
    #comparison_processed_data_reg.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison_region")
    
    return comparison_processed_data_reg

In [ ]:
region_both = region_aggregation(first_ID_df, second_ID_df)
region_both.head()

# Individual Functions

In [23]:
# maybe config argument?
def download_DS_data(ds_ID, jur_level):
    datafiles = config[ds_ID].values()
    
    comparison_no_geozone_df = pd.DataFrame()
    for file_name in datafiles:
        working_df = pd.read_csv(file_name)
        working_df['year'] = f"{file_name[-11:-7]}"
        comparison_no_geozone_df = comparison_no_geozone_df.append(working_df)
        
    # Save the features_first_ID for future use (Used when creating the diff file)
    features = comparison_no_geozone_df.drop(['mgra', 'year'], axis=1).columns
    
    comparison_no_geozone = copy.deepcopy(comparison_no_geozone_df)
    
    # Adding SQl Data to first_id_df
    comparison_processed_data = comparison_no_geozone.merge(jur_level, how='left', on='mgra')
    
    # making it original
    comparison_processed_data.columns = [x + f'_{ds_ID}' for x in comparison_processed_data.columns]
    
    #comparison_first_ID_processed_data.to_csv(f"{output_path}/comparison_{first_ID}_processed_data.csv")
    
    return comparison_processed_data, comparison_no_geozone_df, features

In [24]:
first_ID_processed, first_ID_unprocessed, first_ID_features = download_DS_data(first_ID, jur_level)

## CPA Aggregation

In [35]:
def cpa_aggregation_ind(first_ID_df, cpa_level):
    
    # Adding SQl Data (CPA) to first_id_df
    comparison_first_ID_processed_data_cpa = first_ID_df.merge(cpa_level, how='left', on='mgra')
    comparison_first_ID_processed_data_cpa = comparison_first_ID_processed_data_cpa[comparison_first_ID_processed_data_cpa['geozone'] != '*Not in a CPA*']

    # Drop the MGRA column because it isn't really a quantitative value
    comparison_processed_data_cpa = comparison_first_ID_processed_data_cpa.drop('mgra', axis=1)

    # Aggregate the sum of features by geozone and year
    comparison_processed_data_cpa = comparison_processed_data_cpa.groupby(['geozone', 'year']).sum()

    # Rename index (geozone -> cpa)
    comparison_processed_data_cpa.index.names = ['cpa', 'year']
    
    #comparison_processed_data_cpa.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison_cpa")

    return comparison_processed_data_cpa

In [36]:
cpa_aggregation_ind(first_ID_unprocessed, cpa_level)

taz   hs  hs_sf  hs_mf  hs_mh   hh  hh_sf  \
cpa                       year                                                
32nd Street Naval Station 2016  86368  161     60    101      0  161     60   
                          2018  86368  163     62    101      0  162     61   
                          2020  86368  163     62    101      0  162     61   
                          2023  86368  163     62    101      0  162     61   
                          2025  86368  163     62    101      0  162     61   
...                               ...  ...    ...    ...    ...  ...    ...   
Via De La Valle           2032  55759  499    383    116      0  462    353   
                          2035  55759  500    384    116      0  462    353   
                          2040  55759  508    392    116      0  464    355   
                          2045  55759  508    392    116      0  464    355   
                          2050  55759  509    393    116      0  465    356   

                                hh_mf  hh_mh  gq_civ  ...  midpriceroom  \
cpa                       year                        ...                 
32nd Street Naval Station 2016    101      0       0  ...             0   
                          2018    101      0       0  ...             0   
                          2020    101      0       0  ...             0   
                          2023    101      0       0  ...             0   
                          2025    101      0       0  ...             0   
...                               ...    ...     ...  ...           ...   
Via De La Valle           2032    109      0       0  ...             0   
                          2035    109      0       0  ...             0   
                          2040    109      0       0  ...             0   
                          2045    109      0       0  ...             0   
                          2050    109      0       0  ...             0   

                                upscaleroom  hotelroomtotal  luz_id  \
cpa                       year                                        
32nd Street Naval Station 2016            0              25    2538   
                          2018            0              28    2538   
                          2020            0              28    2538   
                          2023            0              29    2538   
                          2025            0              29    2538   
...                                     ...             ...     ...   
Via De La Valle           2032            0               0    2422   
                          2035            0               0    2422   
                          2040            0               0    2422   
                          2045            0               0    2422   
                          2050            0               0    2422   

                                truckregiontype  district27  milestocoast  \
cpa                       year                                              
32nd Street Naval Station 2016               20         200       56.2544   
                          2018               20         200       56.2544   
                          2020               20         200       56.2544   
                          2023               20         200       56.2544   
                          2025               20         200       56.2544   
...                                         ...         ...           ...   
Via De La Valle           2032               31          62       49.6138   
                          2035               31          62       49.6138   
                          2040               31          62       49.6138   
                          2045               31          62       49.6138   
                          2050               31          62       49.6138   

                                     acres  effective_acres  land_acres  
cpa                       year

## Jurisdiction level Data

In [37]:
def jur_aggregation_ind(first_ID_df, jur_level):
    # Adding SQl Data (Jurisdiction) to first_id_df
    comparison_first_ID_processed_data_jur = first_ID_df.merge(jur_level, how='left', on='mgra')
    
    # Drop the MGRA column because it isn't really a quantitative value
    comparison_processed_data_jur = comparison_first_ID_processed_data_jur.drop('mgra', axis=1)
    
    # Aggregate the sum of features by geozone and year
    comparison_processed_data_jur = comparison_processed_data_jur.groupby(['geozone', 'year']).sum()
    
    # Rename index (geozone -> jurisdiction)
    comparison_processed_data_jur.index.names = ['jurisdiction', 'year']
    
    #comparison_processed_data_jur.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison_jurisdiction")
    
    return comparison_processed_data_jur

In [39]:
jur_aggregation_ind(first_ID_unprocessed, jur_level)

taz     hs  hs_sf  hs_mf  hs_mh     hh  hh_sf  hh_mf  \
jurisdiction year                                                            
Carlsbad     2016  870633  50670  34815  14215   1640  47670  32817  13351   
             2018  870633  51501  35865  13994   1642  48718  33985  13229   
             2020  870633  52666  36426  14598   1642  49783  34559  13719   
             2023  870633  53363  36634  15087   1642  50383  34761  14117   
             2025  870633  53899  36987  15270   1642  50868  35050  14313   
...                   ...    ...    ...    ...    ...    ...    ...    ...   
Vista        2032  510150  42133  24218  15730   2185  40496  23326  15034   
             2035  510150  43101  24488  16428   2185  41370  23575  15659   
             2040  510150  43887  25012  16690   2185  41749  23752  15861   
             2045  510150  43988  25043  16760   2185  41868  23797  15935   
             2050  510150  44071  25072  16814   2185  42002  23846  16020   

                   hh_mh  gq_civ  ...  midpriceroom  upscaleroom  \
jurisdiction year                 ...                              
Carlsbad     2016   1502     915  ...           940          765   
             2018   1504     915  ...          1026          837   
             2020   1505     915  ...          1026          837   
             2023   1505     915  ...          1056          862   
             2025   1505     915  ...          1076          879   
...                  ...     ...  ...           ...          ...   
Vista        2032   2136    2327  ...           130          115   
             2035   2136    2327  ...           132          117   
             2040   2136    2327  ...           135          120   
             2045   2136    2327  ...           137          122   
             2050   2136    2327  ...           139          124   

                   hotelroomtotal  luz_id  truckregiontype  district27  \
jurisdiction year                                                        
Carlsbad     2016            3406   21520              811         943   
             2018            3722   21520              811         943   
             2020            3722   21520              811         943   
             2023            3833   21520              811         943   
             2025            3905   21520              811         943   
...                           ...     ...              ...         ...   
Vista        2032             774   30331              869         869   
             2035             789   30331              869         869   
             2040             807   30331              869         869   
             2045             818   30331              869         869   
             2050             833   30331              869         869   

                   milestocoast         acres  effective_acres    land_acres  
jurisdiction year                                                             
Carlsbad     2016     1918.0907  27460.971113     16381.889663  26574.857225  
             2018     1918.0907  27460.971113     16381.889663  26574.857225  
             2020     1918.0907  27460.971113     16381.889663  26574.857225  
             2023     1918.0907  27460.971113     16381.889663  26574.857225  
             2025     1918.0907  27460.971113     16381.889663  26574.857225  
...                         ...           ...              ...           ...  
Vista        2032     6037.2739  16426.080227     14655.309729  16408.292733  
             2035     6037.2739  16426.080227     14655.309729  16408.292733  
             2040     6037.2739  16426.080227     14655.309729  16408.292733  
             2045     6037.2739  16426.080227     14655.309729  16408.292733  
             2050     6037.2739  16426.080227     14655.309729  16408.292733  

[247 rows x 103 columns]

## Region level Data

In [40]:
def region_aggregation_ind(first_ID_df):
    
    # Aggregate the sum of features by year
    comparison_processed_data_reg = first_ID_df.groupby('year').sum()
    
    # Drop the MGRA column because it isn't really a quantitative value
    comparison_processed_data_reg = comparison_processed_data_reg.drop('mgra', axis=1)
    
    #comparison_processed_data_reg.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison_region")
    
    return comparison_processed_data_reg

In [42]:
region_aggregation_ind(first_ID_unprocessed)

,taz,hs,hs_sf,hs_mf,hs_mh,hh,hh_sf,hh_mf,hh_mh,gq_civ,...,midpriceroom,upscaleroom,hotelroomtotal,luz_id,truckregiontype,district27,milestocoast,acres,effective_acres,land_acres
year,,,,,,,,,,,,,,,,,,,,,
2016,53370775,1190554,717626,430716,42212,1134848,687509,407158,40181,63014,...,15396,13062,56646,2431123,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06
2018,53370775,1205852,723209,440536,42107,1147635,692426,415193,40016,69372,...,16823,14280,61917,2431123,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06
2020,53370775,1226462,730452,453903,42107,1166240,698993,427158,40089,72056,...,16823,14280,61917,2431123,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06
2023,53370775,1262957,739359,481491,42107,1197072,705785,451128,40159,73875,...,17308,14694,63707,2431123,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06
2025,53370775,1288217,746460,499650,42107,1219745,711877,467676,40192,74447,...,17627,14970,64889,2431123,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06
2026,53370775,1300847,749930,508810,42107,1231007,714801,475993,40213,74733,...,17764,15084,65379,2431123,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06
2029,53370775,1338737,758815,537815,42107,1264151,722304,501605,40242,75591,...,18183,15439,66938,2431123,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06
2030,53370775,1351367,760811,548449,42107,1274948,724048,510655,40245,75877,...,18320,15553,67428,2431123,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06
2032,53370775,1376162,764402,569653,42107,1296193,727127,528813,40253,76043,...,18546,15738,68233,2431123,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06


# Environment Setup

## Pulling Info From YML File

In [143]:
# Localise with . files 
# config_filename = 'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/ds41_42_config.yml'
config_filename = './ds_config.yml'

In [144]:
with open(config_filename, "r") as yml_file:
    config = yaml.safe_load(yml_file)

In [145]:
# Possible errors: yml syntax errors (make sure your inputs are valid)

# General inputs
check_type = config['inputs']['check_type'].lower()
output_path = config['inputs']['output_path']

In [146]:
# Comparison inputs
first_ID = config['inputs']['first_ID'].upper()
second_ID = config['inputs']['second_ID'].upper()

In [147]:
# Individual inputs
individual_ID = config['inputs']['individual_ID']

## Check Input Errors

In [148]:
# invalid check_type input
if check_type not in ['comparison', 'individual']:
    raise AssertionError("Invalid check_type in yml file. Please use either 'comparison' or 'individual'")
    
if check_type == 'comparison':
    # identical ds_id's
    if first_ID == second_ID:
        raise AssertionError('first_ID and second_ID inputs are equivalent. Please make sure they are different values')
    
    # first_id actually exists in list
    if first_ID not in config.keys():
        raise AssertionError('first_ID not found in specified DS_ID list. Please reference yml file for selectable DS_IDs')
    
    # second_id actually exists in list
    if second_ID not in config.keys():
        raise AssertionError('second_ID not found in specified DS_ID list. Please reference yml file for selectable DS_IDs')
        
elif check_type == 'individual':
    # individual_id actually exists in list
    if individual_ID not in config.keys():
        raise AssertionError('individual_ID not found in specified DS_ID list. Please reference yml file for selectable DS_IDs')

In [149]:
# Pulling comparison ds datafiles
first_ID_datafiles = config[first_ID].values()
second_ID_datafiles = config[second_ID].values()

In [150]:
# Pulling individual ds datafiles
individual_datafiles = config[individual_ID].values()

## Downloading SQL Data

In [151]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DDAMWSQL16.sandag.org;'
                      'Database=demographic_warehouse;'
                      'Trusted_Connection=yes;')

In [152]:
# NOTE: Format to make it select correct MGRA series based on input DS
query_all = "SELECT mgra, geotype, geozone FROM demographic_warehouse.dim.mgra WHERE series = 14 AND (geotype='cpa' OR geotype='jurisdiction')" #Remove the last and part when I do this for real 

In [153]:
sql_query = pd.read_sql_query(query_all,conn)

sql_df_all = pd.DataFrame(sql_query)

In [154]:
# SQl Data at different levels
jur_level = sql_df_all[sql_df_all['geotype']=='jurisdiction'].drop('geotype', axis=1).drop_duplicates()
cpa_level = sql_df_all[sql_df_all['geotype']=='cpa'].drop('geotype', axis=1).drop_duplicates()

# Calling Functions

In [166]:
if check_type == 'comparison':
    # download data for each ds_id
    first_ID_processed, first_ID_unprocessed, first_ID_features = download_DS_data(first_ID, jur_level)
    second_ID_processed, second_ID_unprocessed, second_ID_features = download_DS_data(second_ID, jur_level)
    
    # concatenate both processed ds_id's together to get mgra level data by mgra and year
    mgra_both = concat_dfs(first_ID_processed, second_ID_processed)
    #to_csv (maybe move from function to here)
    
    # aggregate both ds_id's together by cpa level and year
    cpa_both = cpa_aggregation(first_ID_unprocessed, second_ID_unprocessed, cpa_level)
    #to_csv (maybe)
    
    # aggregate both ds_id's together by jurisdiction level and year
    jur_both = jur_aggregation(first_ID_unprocessed, second_ID_unprocessed, jur_level)
    
    # aggregate both ds_id's together by region level (sum total)
    region_both = region_aggregation(first_ID_unprocessed, second_ID_unprocessed)
    
    # Create diff at mgra level
    mgra_diff = create_diff(first_ID_features, second_ID_features, mgra_both)

    # Create diff at cpa level
    cpa_diff = create_diff(first_ID_features, second_ID_features, cpa_both)

    # Create diff at jurisdiction level
    jur_diff = create_diff(first_ID_features, second_ID_features, jur_both)

In [169]:
if check_type == 'individual':
    
    # download data for each ds_id
    individual_ID_processed, individual_ID_unprocessed, individual_ID_features = download_DS_data(individual_ID, jur_level)
    
    # aggregate both ds_id's together by cpa level and year
    cpa_ind = cpa_aggregation_ind(individual_ID_unprocessed, cpa_level)
    #to_csv (maybe)
    
    # aggregate both ds_id's together by jurisdiction level and year
    jur_ind = jur_aggregation_ind(individual_ID_unprocessed, jur_level)

    # aggregate both ds_id's together by region level (sum total)
    region_ind = region_aggregation_ind(individual_ID_unprocessed)

# Brainstorming for Part 2

In [ ]:
config['dof']['dof_data'] # figure out what this is for and what to do with it

In [ ]:
first_second_ID_comparison.head()

In [ ]:
# MGRA level stats
first_second_ID_comparison.describe(percentiles=[0.5])

### Outlier Detection

In [ ]:
# Good article: https://machinelearningmastery.com/how-to-use-statistics-to-identify-outliers-in-data/

In [ ]:
diff_stats = diff_df.describe()

In [ ]:
diff_stats

In [ ]:
from scipy import stats

In [ ]:
first_second_ID_comparison

In [ ]:
q = first_second_ID_comparison["taz_DS35"].quantile(0.99)
first_second_ID_comparison[first_second_ID_comparison["taz_DS35"] > q]

In [ ]:
first_second_ID_comparison[first_second_ID_comparison["taz_DS35"] > q]

In [ ]:
first_second_ID_comparison['taz_DS35']

In [ ]:
first_second_ID_comparison[(np.abs(stats.zscore(first_second_ID_comparison)) < 3).all(axis=1)]

In [ ]:
diff_stats.loc['max']

In [ ]:
diff_stats.loc['max']